In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
from datetime import datetime

from tqdm.notebook import tqdm

%matplotlib notebook

In [2]:
exp_dir = "/Users/romainegele/Documents/Argonne/deephyper-scalable-bo/experiments/polaris/dhb/output/"
# exp_name = "dhb_combo-TPESHA-async-RF-qUCB-qUCB-4-4-3600-42"
# exp_name = "dhb_combo-DBOS4MSHA-async-RF-qUCB-qUCB-4-4-3600-42"
# exp_name = "dhb_combo-DBOS4M-async-RF-qUCB-qUCB-4-4-3600-42"
# exp_name = "dhb_combo-DBOS4MSHA-async-MF-qUCB-qUCB-4-4-3600-42"

# exp_name = "dhb_combo-TPESHA-async-RF-qUCB-qUCB-128-4-10800-42"
# exp_name = "dhb_combo-DBOS4M-async-RF-qUCB-qUCB-128-4-10800-42"
exp_name = "dhb_combo-DBOS4MSHA-async-RF-qUCB-qUCB-128-4-10800-42"

path = os.path.join(exp_dir, exp_name, "results.csv")

df = pd.read_csv(path)

In [3]:
df

,p:activation,p:base_lr,p:batch_normalization,p:batch_size,p:dense_0,p:dense_1,p:dense_2,p:dense_feature_layers_0,p:dense_feature_layers_1,p:dense_feature_layers_2,...,objective,job_id,rank,timestamp_submit,timestamp_gather,timestamp_start,timestamp_end,step,pruned,num_parameters
0,softplus,0.002841,True,117,11,136,175,521,71,74,...,0.065546,0,0,1.295770,37.302210,2.476832,37.301656,1,True,2604959
1,hard_sigmoid,0.006957,False,283,637,43,51,17,817,676,...,-0.073699,0,21,1.867126,29.811767,3.484543,29.810861,1,True,2538862
2,linear,0.000020,True,310,116,91,486,203,26,12,...,-1.000000,0,50,1.225070,30.425059,1.413570,30.423589,1,True,1042343
3,sigmoid,0.001091,True,462,501,260,246,357,118,832,...,-1.000000,0,164,1.208335,30.513057,1.320749,30.512007,1,True,3436005
4,selu,0.000457,True,206,132,138,281,46,15,368,...,-0.831534,0,342,1.288919,30.179831,2.442441,30.178641,1,True,436759
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47173,swish,0.000197,False,62,178,255,135,1006,663,35,...,-0.001626,75,163,10488.177215,10545.268891,10488.185383,10545.268329,1,True,6292669
47174,swish,0.000150,True,157,960,164,13,31,111,621,...,0.623651,109,369,10504.105064,10545.736474,10504.113762,10545.735687,1,True,2243684
47175,softsign,0.001170,False,276,151,913,79,29,558,529,...,-1.000000,115,500,10505.040141,10545.756593,10505.048724,10545.756016,1,True,1214459
47176,swish,0.000024,True,301,360,643,129,839,190,15,...,0.679460,72,278,10481.263711,10546.841135,10481.272869,10546.840499,3,True,4669756


In [4]:
hp_names = [cname for cname in df.columns if "p:" in cname]
hp_names

['p:activation',
 'p:base_lr',
 'p:batch_normalization',
 'p:batch_size',
 'p:dense_0',
 'p:dense_1',
 'p:dense_2',
 'p:dense_feature_layers_0',
 'p:dense_feature_layers_1',
 'p:dense_feature_layers_2',
 'p:dropout',
 'p:early_stopping',
 'p:early_stopping_patience',
 'p:learning_rate',
 'p:loss',
 'p:optimizer',
 'p:reduce_lr',
 'p:reduce_lr_factor',
 'p:reduce_lr_patience',
 'p:residual',
 'p:scaling',
 'p:warmup_lr']

In [5]:
hp_types = [df[k].dtype for k in hp_names]
hp_types

[dtype('O'),
 dtype('float64'),
 dtype('bool'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('float64'),
 dtype('bool'),
 dtype('int64'),
 dtype('float64'),
 dtype('O'),
 dtype('O'),
 dtype('bool'),
 dtype('float64'),
 dtype('int64'),
 dtype('bool'),
 dtype('O'),
 dtype('bool')]

In [6]:
df_str = df.astype({k:str for k in hp_names})
df_str["hash"] = df_str[hp_names].agg('-'.join, axis=1)
df_str = df_str[["hash", "objective", "timestamp_end"]]
df_str

,hash,objective,timestamp_end
0,softplus-0.0028413585465125-True-117-11-136-17...,0.065546,37.301656
1,hard_sigmoid-0.0069570787597008-False-283-637-...,-0.073699,29.810861
2,linear-1.976428084903253e-05-True-310-116-91-4...,-1.000000,30.423589
3,sigmoid-0.0010906038716589-True-462-501-260-24...,-1.000000,30.512007
4,selu-0.0004566767026172-True-206-132-138-281-4...,-0.831534,30.178641
...,...,...,...
47173,swish-0.0001966619830679-False-62-178-255-135-...,-0.001626,10545.268329
47174,swish-0.0001495123090435-True-157-960-164-13-3...,0.623651,10545.735687
47175,softsign-0.0011695701694482-False-276-151-913-...,-1.000000,10545.756016
47176,swish-2.3830980948506262e-05-True-301-360-643-...,0.679460,10546.840499


In [7]:
num_evals = len(df_str)
num_unique_evals = len(df_str["hash"].drop_duplicates())
print(f"#eval:{num_evals} - #unique-evals:{num_unique_evals}")

#eval:47178 - #unique-evals:47178


In [8]:
id_max = df_str["objective"].argmax()
hash_max = df_str.iloc[id_max]["hash"]
objective_max = df_str.iloc[id_max]["objective"]

In [9]:
best_config_repetitions = df_str[df_str["hash"] == hash_max]
num_occurence = len(best_config_repetitions)
min_objective = best_config_repetitions["objective"].min()
max_objective = best_config_repetitions["objective"].max()
print(f"#occ:{num_occurence} - min:{min_objective:.3f} - max:{max_objective:.3f}")

#occ:1 - min:0.920 - max:0.920


In [10]:
earliest_timestamps = (df_str.groupby("hash")
                       .agg({"timestamp_end": "min"})
                       .reset_index()["timestamp_end"]
                       .values)
all_timestamps = df_str["timestamp_end"].values
indices = np.where(np.in1d(earliest_timestamps, all_timestamps))[0]

In [11]:
df_str.loc[:, ("earliest-occurence",)] = 0
df_str.loc[indices, ("earliest-occurence",)] = 1
df_str = df_str.sort_values("timestamp_end")
df_str = df_str.reset_index(drop=True)

In [12]:
df_str.loc[:, ("better-than-baseline",)] = 0
df_str.loc[(df_str["objective"] >= 0.88), ("better-than-baseline",)] = 1
df_str.loc[:, ("count",)] = (df_str["earliest-occurence"] & df_str["better-than-baseline"]).astype(int)
df_str.loc[:, ("count",)] = df_str["count"].cumsum()
df_str

,hash,objective,timestamp_end,earliest-occurence,better-than-baseline,count
0,hard_sigmoid-0.0069570787597008-False-283-637-...,-0.073699,29.810861,1,0,0
1,selu-0.0004566767026172-True-206-132-138-281-4...,-0.831534,30.178641,1,0,0
2,linear-1.976428084903253e-05-True-310-116-91-4...,-1.000000,30.423589,1,0,0
3,sigmoid-0.0010906038716589-True-462-501-260-24...,-1.000000,30.512007,1,0,0
4,swish-0.0003894922591863-True-354-131-848-28-6...,-0.027499,31.088678,1,0,0
...,...,...,...,...,...,...
47173,swish-0.0001495123090435-True-157-960-164-13-3...,0.623651,10545.735687,1,0,3386
47174,softsign-0.0011695701694482-False-276-151-913-...,-1.000000,10545.756016,1,0,3386
47175,swish-2.3830980948506262e-05-True-301-360-643-...,0.679460,10546.840499,1,0,3386
47176,relu-3.072028252773362e-05-True-210-417-123-14...,0.593661,10547.265661,1,0,3386
